In [1]:
class Action:
    """
    Represents an action that can be applied to a state in a planning problem.
    """

    def __init__(self, name, preconditions, effects, cost):
        """
        Initializes an Action object.

        Args:
        name (str): The name of the action.
        preconditions (dict): The preconditions that must be met for the action to be applicable.
        effects (dict): The effects of the action on the state.
        cost (int): The cost of performing the action.
        """
        self.name = name
        self.preconditions = preconditions  # Preconditions (conditions that must be true for the action to apply)
        self.effects = effects  # Effects (changes in the state after the action is applied)
        self.cost = cost  # Cost of performing the action

    def applicable(self, state):
        """
        Checks if the action is applicable in the current state.

        Args:
        state (dict): The current state.

        Returns:
        bool: True if the action can be applied in the current state, False otherwise.
        """
        # Checks if all preconditions are met in the current state
        return all(state.get(k) == v for k, v in self.preconditions.items())

    def apply(self, state):
        """
        Applies the action to the current state and returns the new state.

        Args:
        state (dict): The current state.

        Returns:
        dict: The new state after applying the action.
        """
        new_state = state.copy()  # Create a copy of the current state
        for effect, value in self.effects.items():
            new_state[effect] = value  # Apply the effects to the new state
        return new_state


class AStarPlanner:
    """
    A class that implements the A* planning algorithm to find a sequence of actions
    to reach a goal state from an initial state.
    """

    def __init__(self, actions, initial_state, goal_state):
        """
        Initializes an AStarPlanner object.

        Args:
        actions (list): A list of available actions.
        initial_state (dict): The initial state from which the planning begins.
        goal_state (dict): The target state we are trying to reach.
        """
        self.actions = actions  # Available actions
        self.initial_state = initial_state  # The starting state
        self.goal_state = goal_state  # The goal state

    def heuristic(self, state):
        """
        Heuristic function: calculates how many goal conditions are not yet satisfied.

        Args:
        state (dict): The current state.

        Returns:
        int: The number of unsatisfied goal conditions.
        """
        # Counts how many goal conditions are not yet met
        return sum(1 for k, v in self.goal_state.items() if state.get(k) != v)

    def plan(self):
        """
        Implements the A* algorithm to plan a sequence of actions to reach the goal.

        Returns:
        list: A list of action names that leads to the goal state, or None if no plan is found.
        """
        open_list = [(0 + self.heuristic(self.initial_state), self.initial_state, [])]  # (f_cost, state, path)
        closed_list = []  # List of visited states

        while open_list:
            open_list.sort()  # Sort open_list by the f_cost (lowest first)
            f_cost, current_state, path = open_list.pop(0)

            # If we have reached the goal state
            if all(current_state.get(k) == v for k, v in self.goal_state.items()):
                return path  # Return the sequence of actions that lead to the goal

            closed_list.append(tuple(current_state.items()))  # Mark the current state as visited

            # Explore the possible actions
            for action in self.actions:
                if action.applicable(current_state):  # Check if the action is applicable to the current state
                    new_state = action.apply(current_state)  # Apply the action to the current state
                    if tuple(new_state.items()) not in closed_list:  # Avoid revisiting the same state
                        g_cost = len(path) + action.cost  # Calculate the cost to reach the new state
                        f_cost = g_cost + self.heuristic(new_state)  # Total cost (g_cost + heuristic)
                        open_list.append((f_cost, new_state, path + [action.name]))  # Add the new state to open_list

        return None  # If no plan was found


#! This can be modified to solve other planning problems

# Define actions
actions = [
    Action('Pick up box', {'at_location': 'floor'}, {'at_location': 'hand'}, 1),  # Action to pick up the box
    Action('Move to table', {'at_location': 'hand', 'next_to': 'table'}, {'at_location': 'table'}, 1),  # Move the box to the table
    Action('Place box on table', {'at_location': 'table'}, {'at_location': 'none'}, 1)  # Place the box on the table
]

# Define initial and goal states
initial_state = {'at_location': 'floor', 'next_to': 'table'}  # The box starts on the floor, next to the table
goal_state = {'at_location': 'none'}  # The goal is to have the box no longer in any location

# Create an A* planner and get the plan
planner = AStarPlanner(actions, initial_state, goal_state)
plan = planner.plan()

print("Plan to reach goal:", plan)  # Print the sequence of actions to reach the goal


Plan to reach goal: ['Pick up box', 'Move to table', 'Place box on table']


In [2]:
def test_planner():
    """
    Test the A* planner to ensure it generates the correct plan for reaching the goal state.
    It checks both a normal case and the case where the goal is already reached.
    """

    # Define actions
    actions = [
        Action('Pick up box', {'at_location': 'floor'}, {'at_location': 'hand'}, 1),  # Action to pick up the box
        Action('Move to table', {'at_location': 'hand', 'next_to': 'table'}, {'at_location': 'table'}, 1),  # Action to move the box to the table
        Action('Place box on table', {'at_location': 'table'}, {'at_location': 'none'}, 1)  # Action to place the box on the table
    ]

    # Define initial and goal states
    initial_state = {'at_location': 'floor', 'next_to': 'table'}  # The box starts on the floor, next to the table
    goal_state = {'at_location': 'none'}  # The goal is to have the box no longer in any location

    # Create the planner object
    planner = AStarPlanner(actions, initial_state, goal_state)

    # Execute the planning function to get the plan
    plan = planner.plan()

    # Test that the plan is not None and that it contains the expected sequence of actions
    assert plan is not None, "Plan should not be None"  # Ensure a plan is returned
    assert plan == ['Pick up box', 'Move to table', 'Place box on table'], f"Expected plan, but got {plan}"  # Check the correct sequence of actions

    # Test where the goal is already achieved (no actions should be needed)
    initial_state_already_goal = {'at_location': 'none', 'next_to': 'table'}  # The box is already at the goal location
    planner = AStarPlanner(actions, initial_state_already_goal, goal_state)  # Create a new planner with the goal already achieved
    plan = planner.plan()  # Execute the planning function

    # Test that the plan is empty since the goal is already reached
    assert plan == [], "Plan should be empty since the goal is already achieved"  # Ensure no actions are returned

    print("All tests passed!")  # If all assertions pass, print that all tests have passed


# Execute the tests
test_planner()

All tests passed!
